### IGV TE insertions snapshots

We want to have an outmated way of generating IGV screenshots for the TE insertions. Then, we will use the screenshots for visual confirmation of TE insertions and transduction calls predicted by TraFic. 

**Steps**:
1. Script to produce a bed file with the TE insertion coordinates from TraFic.
2. Script to generate IGV batch file for running IGV
3. Execute IGV with the batch file to produce the screenshots

### 1. Script to produce a bed file with the TE insertion and transduced region coordinates

This script will take as input the TraFiC somatic TE insertions. Format:

1. Project Id
2. Donor Id
3. Tumour Id
4. Chr (insertion)
5. End + cluster (insertion)
6. Beginning - cluster (insertion)
7. Family TE (L1, ALU, OTHER (SVA), ERVK)
8. Gene affected
9. Insertion type. Tdo0(solo), Td1 (partnered transduccion), Td2 (Orphan transduction)
10. Chr (source element)
11. Pos1 (source element)
12. Pos2 (source element)
13. Orientation (+, - or putative if strand was not determined) (source element)
14. Pos1 (transduced region. In Td1 only one, since the beginning will be the end of the source element. Position 1 or 2 depending on the strand of the source element)
15. Pos2 (transduced region. In Td1 only one. Same as in Pos1)
16. Distance to source element (- if putative)
17. Size transduced region
18. Number of reads supporting + cluster
19. Number of reads supporting - cluster

For a Td0, Td1 and Td2 we need to produce a bed entry containing the TE insertion coordinates. 

Bed format:
1. chr - 4th field in somatic 
2. chrStart - 5th field in somatic 
3. chrEnd - 6th field in somatic
4. name - Composed by *insertionBkp-7th-9th:4th_5th_6th*

Additionally, for the Td1 and Td2 we need to produce a bed entry containing the transduced region coordinates. These coordinates will vary depending on the situation:

* Td1 and Td 2 (Source element strand determined)
    * Plus Strand
        * chr - 10th field in somatic
        * beg - Pos1 source element (11th)
        * end - Pos2 transduced region (15th)
        * name - Composed by *trasducedRegion-7th-9th:10th_11th_15th*
        
    * Minus Strand
        * chr - 10th field in somatic
        * beg - Pos1 transduced region (14th)
        * end - Pos2 source element (12th)
        * name - Composed by *trasducedRegion-7th-9th:10th_14th_12th*
    
* Td1 or Td0 (Strand do not determined, putative)
    * chr - 10th field in somatic
    * beg - Pos1 transduced region (14th) - margin (5.000bp default)
    * end - Pos2 transduced region (15th) + margin (5.000bp default)
    * name - Composed by *trasducedRegion-7th-9th:10th_11th_12th*
    

In [10]:
# User´s input 
inputFile = "/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt"
name = "4283773c-c04b-4f33-a398-f3566f6d293a" 
margin = 5000
outDir = "/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2"

## Open input and output files
somaticInsertions=open(inputFile, 'r')

fileName = name + ".bed"
outFilePath = outDir + "/" + fileName
outFile = open( outFilePath, "w" )

# Read input file line by line
for line in somaticInsertions:
    line = line.rstrip('\n')
    line = line.split("\t")
    projectId = line[0]
    donorId = line[1]
    tumourId = line[2]
    insertChr = line[3]
    insertBeg = line[4]
    insertEnd = line[5]
    family = line[6]
    gene = line[7]
    insertType = line[8]
    sourceChr = line[9]
    sourcePos1 = line[10]
    sourcePos2 = line[11]
    strand = line[12]
    transducedPos1 = line[13]
    transducedPos2 = line[14]
    dist2source = line[15]
    size = line[16]
    nbReadsPlus = line[17]
    nbReadsMinus = line[18]

    # Discard donors without any transposable element (TE) insertion 
    if family != "empty":  
        
        # Report TE insertion coordinates in bed format
        featureName = "insertionBkp" + "-" + family + "-" + insertType + ":" + insertChr + "_" + insertBeg + "_" + insertEnd  
        row = insertChr + "\t" + insertBeg + "\t" + insertEnd + "\t" + featureName + "\n"
        outFile.write(row)
        
        # Additionally, report transduction coordinates if it is a Td1 
        # (partnered transduccion) or a Td2 (Orphan transduction)
        # Coordinates will depend on the orientation of the TE
        if insertType == "td1" or insertType == "td2":
            
            # A) TE orientation unknown:   
            # ---source?---  ---TransducedRegion---  ---source?---
            if strand == "putative":
                beg = int(transducedPos1) - margin
                end = int(transducedPos2) + margin
            
            # B) TE in positive strand
            # ---source---  ---TransducedRegion---  
            elif strand == "plus":
                beg = sourcePos1
                end = transducedPos2
            
            # C) TE in negative strand
            # ---TransducedRegion---  ---source---
            else:
                beg = transducedPos1
                end = sourcePos2
            
            # Report the coordinates
            featureName = "transducedRegion" + "-" + family + "-" + insertType + ":" + insertChr + "_" + insertBeg + "_" + insertEnd 
            row = sourceChr + "\t" + str(beg) + "\t" + str(end) + "\t" + featureName + "\n"
            outFile.write(row)
        
# Close output and end
outFile.close()


### 2. Script to generate an IGV batch file for running IGV

This script will take as input a bed file with the Trafic insertion coordinates and a bam file plus other optional parameters: 

* **Mandatory**
    * Input bed
    
        1. chr 
        2. chrStart 
        3. chrEnd 
        4. name 
    
    * Input bam


* **Optional**
    * Genome Version. Default: hg19
    * Track in bed format. E.g: repeat masker repeats. 
    * Margin. Number of bases around the range defined in the bed entry to be included in the snapshot. 
    * maxPanelHeight. Sets the number of vertical pixels (height) of each panel to include in image. Default: 1000
    * Format. Image file format, which must be png, jpg, or svg. Default: png
    * OutDir. Default: current working directory. 


It will produce as output an IGV batch file like this one:

```
new
genome hg19
load /Users/brodriguez/Research/Projects/Pancancer/Igv_snapshots/db2a1df8-487d-4dad-a347-5c5ed539d5ad.bam
load /Users/brodriguez/Research/Projects/Pancancer/Igv_snapshots/repeatmasker_data_from_ucsc_090112.sorted.bed
snapshotDirectory /Users/brodriguez/Research/Projects/Pancancer/Igv_snapshots 
goto chrX:19532036-19534134
snapshot 
goto chrX:39964716-39966767
snapshot 
goto chr20:38234854-38236804
snapshot
exit
```


In [8]:
# User´s input 
inputBed = "/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/4283773c-c04b-4f33-a398-f3566f6d293a.bed"
inputBam = "/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/4283773c-c04b-4f33-a398-f3566f6d293a.bam"
fileName = "igv_batch.txt"
genome = "hg19"
track = "/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test1/repeatmasker_data_from_ucsc_090112.sorted.bed"
margin = "1000"
maxPanelHeight = "500"
imgFormat = "png" 
outDir = "/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test3"

## Open input and output files
bed = open(inputBed, 'r')

outFilePath = outDir + "/" + fileName
outFile = open( outFilePath, "w" )

## Add basic tasks (create session, load genome, bam, optional track...) to batch file
outFile.write("%s\n" % "new")
outFile.write("%-1s %s\n" % ("genome", genome))
outFile.write("%-1s %s\n" % ("load", inputBam))

if track != "none":
    outFile.write("%-1s %s\n" % ("load", track))    

## Iterate over the input bed file and create the tasks to take snapshots
# of the regions of interest:
for line in bed:
    line = line.rstrip('\n')
    line = line.split("\t")
    chrom = line[0]
    beg = int(line[1]) - int(margin) 
    end = int(line[2]) + int(margin)
    snapshotPos = "chr" + str(chrom) + ":" + str(beg) + "-" + str(end)
    snapshotName = line[3] + "." +  imgFormat
    
    # Write tasks to take the snapshots into the batch file 
    outFile.write("%-1s %s\n" % ("goto", snapshotPos))
    outFile.write("%-1s %s\n" % ("maxPanelHeight", maxPanelHeight))
    outFile.write("%-1s %s\n" % ("snapshot", snapshotName))

# Add exit task
outFile.write("%s\n" % "exit")

# Close batch file and ends
outFile.close()
    

### 2. Execute IGV with the batch file to produce the screenshots

### 2.1 Test 1 files

I will execute the IGV with the batch file:

```
igv=/Users/brodriguez/Research/Apps/IGV/2.3.72/igv.sh
batchFile=/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test1/igv_batch.txt
outDir=/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test1/Screenshots/
cd $outDir

time $igv -g hg19 -b $batchFile
real: 41m3.118s
user: 2m9.247s
sys: 0m10.458s
```

Images look good. It took ~41' to generate the 532 imagess corresponding to the 532 insertions in one donor

Generate the images for the transduced regions for the 2 td2 insertions we have:

```
igv=/Users/brodriguez/Research/Apps/IGV/2.3.72/igv.sh
batchFile=/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test1/igv_batch_transducedRegions.txt
outDir=/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test1/Screenshots/
cd $outDir

time $igv -g hg19 -b $batchFile
```

The images looks good.

### 2.2 Test 2 files
As test1 donor did not contain many transductions I asked jose for the minibam from another donor. 

He sent me one with 19 td1 and 61 td2. I will execute the IGV on the already generated batch file:

```
igv=/Users/brodriguez/Research/Apps/IGV/2.3.72/igv.sh
batchFile=/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/igv_batch.txt
outDir=/Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/Screenshots/
cd $outDir

time $igv -g hg19 -b $batchFile
real	52m58.189s
user	2m44.327s
sys	0m14.706s
```

#### Pick an example of each type to verify the image is fine: 
* Td0 (solo-L1)

```
grep 'td0' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | head -1
ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	X	5593350	5593301	L1	RP11-733O18.1	td0	-	-	-	-	-	-	-	7

cd /Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/Screenshots
open insertionBkp-L1-td0:X_5593350_5593301.png
Ok, it looks good, multicolored positive and negative clusters
```
    
* Td1 (partnered-transduccion)
    * putative
    
    ```
    grep 'td1' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | grep 'putative' | head -1
    ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	1	37673028	37672993	L1	-	td1	4	117340987	117341153	putative 117341050	117341217	-	-	5	10	5
    
    cd /Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/Screenshots
    
    open insertionBkp-L1-td1:1_37673028_37672993.png
    Ok, looks good, single colored positive and multicolored negative cluster
    
    open transducedRegion-L1-td1:1_37673028_37672993.png
    Ok, looks good, multicolored cluster with a subset of reads with mates on the same region (blue reads). Putative source L1PA up or down stream
    ```
  
    * plus
    ```
    grep 'td1' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | grep 'plus' | head -1
    ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	X	146016754	146016705	L1	-	td1	22	29059271	29065303	plus	29065497	194	-	19	18	19
    
    open insertionBkp-L1-td1:X_146016754_146016705.png
    Ok, weird.. two colored positive and multicolored negative cluster. Ask José
    
    open transducedRegion-L1-td1:X_146016754_146016705.png
    Ok, looks good, cluster composed by set of reads of the same colours (they do correspond to different transduction events). Source L1H upstream
    ```
    
    * minus
    ```
    grep 'td1' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | grep 'minus' | head -1
    ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	7	145833301	145833294	L1	CNTNAP2	td1	4	137214649	137220701	minus	137213866	-	783	-	7	6	7
    
    open insertionBkp-L1-td1:7_145833301_145833294.png
    Ok, multicolored positive and singlecolored negative cluster
    
    open transducedRegion-L1-td1:7_145833301_145833294.png
    Ok, looks good, cluster composed by set of reads of the same colours (they do correspond to different transduction events). Source L1H downstream
    ```
    
* Td2 (orphan-transduccion)
    * putative
    ```
    grep 'td2' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | grep 'putative' | head -1
    ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	X	58154067	58154025	L1	-	td2	6	13190686	13190980	putative 13191016	13191505	-	489	9	10	9
    
    cd /Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/Screenshots
    
    open insertionBkp-L1-td2:X_58154067_58154025.png
    Ok, looks good, single colored positive and negative clusters
    
    open transducedRegion-L1-td2:X_58154067_58154025.png
    Ok, looks good. Transduction cluster + somatic source element cluster
    ```
  
    * plus
    ```
    grep 'td2' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | grep 'plus' | head -1
    ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	2	36358155	36358126	L1	-	td2	20	23406745	23412777	plus	23412963	23413658	881	695	4	4	4
    
    cd /Users/brodriguez/Research/Projects/Pancancer/Somatic/Analysis/Igv_snapshots/Test2/Screenshots
    
    open insertionBkp-L1-td2:2_36358155_36358126.png
    Ok, looks good, single colored positive and negative clusters
    
    open transducedRegion-L1-td2:2_36358155_36358126.png
    Ok, looks good, cluster composed by set of reads of the same colours (they do correspond to different transduction events). Source L1H upstream
    ```
    
    * minus
    ```
    grep 'td2' 4283773c-c04b-4f33-a398-f3566f6d293a_somaticInsertions.txt | grep 'minus' | head -1
    ESAD-UK	4283773c-c04b-4f33-a398-f3566f6d293a	OCCAMS-AH-091	X	9823549	9823486	L1	SHROOM2	td2	4	137214649	137220701	minus	137213864	137214653	785	789	7	4	7
    
    open insertionBkp-L1-td2:X_9823549_9823486.png
    Ok, looks good, single colored positive and negative clusters
    
    open transducedRegion-L1-td2:X_9823549_9823486.png
    Ok, looks good, cluster composed by set of reads of the same colours (they do correspond to different transduction events). Source L1H downstream
    ```


**Ok, as everything works fine I will put the code into two scripts**



